<a href="https://colab.research.google.com/github/ThisGuy245/AI-Tasks/blob/main/AI_Task_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:


import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
import difflib  # For string matching

huggingface-cli login

# Load the model and tokenizer
model_name = "llama-3.3"
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.3-70B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.3-70B-Instruct")

# Function to generate a fluent response with faster response times
def generate_response(prompt, max_new_tokens=60, temperature=0.7, top_k=40):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    start_time = time.time()
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_k=top_k,
        no_repeat_ngram_size=2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    end_time = time.time()
    response_time = end_time - start_time
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    ai_response = response.split("AI:")[-1].strip()
    return ai_response, response_time

# Function to handle opening questions
def handle_opening_questions(user_input):
    opening_phrases = {
        "hello": "Hello! I am an AI who talks about more of my kind! Ask me questions about AI.",
        "hi": "Hi there! I'm an AI focused on AI. Feel free to ask me anything about AI advancements.",
        "who are you": "I am an AI who specializes in AI! Ask me questions about AI and its developments.",
        "what do you do": "I talk about AI, its progress, and its impact. Ask me anything related to AI!",
        "how are you": "I'm an AI, so I don't have feelings, but thanks for asking! Ask me about AI advancements!"
    }

    user_input = user_input.lower()
    for key, response in opening_phrases.items():
        if key in user_input:
            return response
    return None

# Function to check if input contains relevant AI-related keywords
def is_relevant_to_ai(user_input):
    ai_keywords = ['ai', 'artificial intelligence', 'machine learning', 'deep learning', 'neural network', 'language model', 'robotics', 'natural language processing', 'computer vision', 'reinforcement learning', 'data science']

    # Check if any AI-related keyword is in the user's input
    return any(keyword in user_input.lower() for keyword in ai_keywords)

# Function to dynamically adjust context and retain important info
def update_context(user_input, ai_response, conversation_history, max_context_length=1000):
    important_info = extract_important_info(user_input)
    conversation_history += f"\nUser: {user_input}\nAI: {ai_response}\n"

    # Retain key information for future context (you can refine the extraction logic)
    if len(conversation_history) > max_context_length:
        conversation_history = conversation_history[-max_context_length:]

    return conversation_history, important_info

# Function to extract important information from the user's input (simplified for example)
def extract_important_info(user_input):
    # Here, you can add more complex logic for extracting important context (e.g., named entities)
    if "name" in user_input:
        return "user_name"
    return ""

# FAQ for more detailed responses
faqs = {
    "What is AI?": "Artificial Intelligence (AI) refers to computer systems designed to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation.",

    "How does machine learning work?": "Machine learning is a subset of AI that focuses on algorithms that can learn from and make predictions or decisions based on data. It involves training models on large datasets to recognize patterns and make informed decisions.",

    "What are the latest advancements in AI?": "Recent AI advancements include improvements in natural language processing, computer vision, reinforcement learning, and generative AI. Specific examples include GPT-3 for language tasks, AlphaFold for protein structure prediction, and DALL-E for image generation.",

    "What are the ethical concerns surrounding AI?": "Key ethical concerns in AI include bias in algorithms, privacy issues, job displacement, autonomous weapons, and the potential for AI to be used for surveillance or manipulation. Ensuring responsible AI development is a major focus in the field.",

    "How is AI impacting different industries?": "AI is transforming industries like healthcare (diagnosis and drug discovery), finance (fraud detection and algorithmic trading), transportation (autonomous vehicles), and entertainment (personalized content recommendations).",

    "What is the future of AI?": "The future of AI may include more advanced general AI systems, improved human-AI collaboration, breakthroughs in quantum AI, and potentially the development of artificial general intelligence (AGI).",
}

# Function to check if user input matches any FAQ or is similar enough to be treated as a match
def find_similar_faq(user_input):
    # Find the FAQ question that most closely matches the user input
    highest_similarity = 0
    best_match = None

    # Use difflib's SequenceMatcher for finding similar strings
    for question in faqs:
        similarity = difflib.SequenceMatcher(None, user_input.lower(), question.lower()).ratio()
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = question

    # If the similarity is above a certain threshold (e.g., 0.8), return the corresponding FAQ answer
    if highest_similarity >= 0.8:
        return faqs[best_match]
    return None

# Main conversation loop
context = "You are an AI assistant specializing in discussing AI advancements. Provide informative and coherent responses about AI progress, its impact on society, and potential future developments."
conversation_history = ""

print("\n\n\nWelcome to the AI Chat! Let's discuss AI advancements.")
print("You can start chatting. Type 'exit' to end the conversation.")

while True:
    user_input = input("User: ")
    if user_input.lower() == 'exit':
        break

    # Handle common opening questions
    opening_response = handle_opening_questions(user_input)
    if opening_response:
        print(f"AI: {opening_response}")
        print("Response time: 0.00 seconds (Opening Question)")
        conversation_history += f"\nUser: {user_input}\nAI: {opening_response}\n"
    else:
        # First check for similar FAQ questions
        faq_response = find_similar_faq(user_input)
        if faq_response:
            print(f"AI: {faq_response}")
            print("Response time: 0.00 seconds (FAQ)")
            conversation_history += f"\nUser: {user_input}\nAI: {faq_response}\n"
        elif is_relevant_to_ai(user_input):
            # Generate AI response if input is AI-related
            prompt = f"{context}\n{conversation_history}\nUser: {user_input}\nAI:"
            ai_response, response_time = generate_response(prompt, max_new_tokens=60, temperature=0.7, top_k=40)

            print(f"AI: {ai_response}")
            print(f"Response time: {response_time:.2f} seconds")
            conversation_history, important_info = update_context(user_input, ai_response, conversation_history)
        else:
            # If the input is not relevant to AI, ask to stay on topic
            print("AI: I like to stay on topic... Let's talk about AI!")
            conversation_history += f"\nUser: {user_input}\nAI: I like to stay on topic... Let's talk about AI!\n"

    # Optionally, log or process the retained important information
    if important_info:
        print(f"Retained information: {important_info}")

print("Thank you for chatting! Here's a summary of our conversation:")
print(conversation_history)


SyntaxError: invalid syntax (<ipython-input-3-5640fa029a5e>, line 1)